In [1]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(1, os.path.join(sys.path[0], "../00_helpers"))
from helpers import get_rice_index

In [2]:
BOLD = '\033[1m'
CLEAR = '\033[0m'
AGE_THRESHOLD = 48

In [3]:
df = pd.read_csv("../processed_data/votes_finaux.csv")
dfp = pd.read_csv("../processed_data/parlementaires_with_age.csv")
dfp.rename(columns={"Unnamed: 0.1.1": "id"}, inplace=True)

In [4]:
# Helper
def get_splitted_share(group_a, group_b, vote_column="vote"):
    """Get share of “yes” votes for two groups"""
    len_yes_a = len(group_a[group_a[vote_column] == "Oui"])
    len_yes_b = len(group_b[group_b[vote_column] == "Oui"])
    len_yesno_a = len(group_a[group_a[vote_column].isin(["Oui", "Non"])])
    len_yesno_b = len(group_b[group_b[vote_column].isin(["Oui", "Non"])])
    share_yes_a = len_yes_a / len_yesno_a * 100
    share_yes_b = len_yes_b / len_yesno_b * 100
    return {
        f"Oui >{AGE_THRESHOLD}": len_yes_a,
        f"Oui <={AGE_THRESHOLD}": len_yes_b,
        f"Oui/Non >{AGE_THRESHOLD}": len_yesno_a,
        f"Oui/Non <={AGE_THRESHOLD}": len_yesno_b,
        f"Oui >{AGE_THRESHOLD} (%)": share_yes_a,
        f"Oui <={AGE_THRESHOLD} (%)": share_yes_b,
        "Différence (%)": abs(share_yes_a - share_yes_b),
    }

### Select political parties with at least 5 councillors

In [5]:
party_count = pd.DataFrame(dfp["party"].value_counts())
party_count.columns = ["count"]
selected_parties = party_count[party_count["count"] > 4]

In [6]:
# ! Doublons parmi les votes finaux
df[df.AffairTitle.duplicated()].shape

(49, 318)

In [7]:
# En cas de doublon, pour le moment, on ne garde que le dernier vote
df = df.sort_values("VoteDate").drop_duplicates(
    subset="AffairTitle", keep="last"
).set_index("AffairTitle").copy()
len(df)

303

## Get votes with largest young/old gap

In [8]:
party_delta = {}
for party_name in selected_parties.index:
    councillor_ids_old = [
        str(i) for i in dfp[(dfp["party"] == party_name) & (dfp["Age"] > 48)]["id"]
    ]
    councillor_ids_young = [
        str(i) for i in dfp[(dfp["party"] == party_name) & (dfp["Age"] <= 48)]["id"]
    ]

    print(
        f"\n{BOLD}{party_name}{CLEAR}:\n {len(councillor_ids_old)} >= 48\n {len(councillor_ids_young)} < 48"
    )

    if len(councillor_ids_young) > 0:
        df_votes_old = df[councillor_ids_old].T.copy()
        df_votes_young = df[councillor_ids_young].T.copy()

        votes = []
        for col in df_votes_young.columns:
            len_votes = len(df_votes_young[df_votes_young[col].isin(["Oui", "Non"])])
            if len_votes == 0:
                # print("No yes/no vote for", col)
                pass
            else:
                vote_share = get_splitted_share(df_votes_old, df_votes_young, vote_column=col)
                vote_share["Date"] = df.T[col]["VoteDate"]
                vote_share["Objet"] = col
                votes.append(vote_share)
        dfv = pd.DataFrame(votes)
        print(f"\nTop 2 objects with largest gap:")
        dfv.sort_values("Différence (%)", ascending=False).head(5).apply(
            lambda row: print(
                f"Delta: {round(row['Différence (%)'], 1)} percent points, %yes <48: {round(row[f'Oui <={AGE_THRESHOLD} (%)'])}% ({row[f'Oui <={AGE_THRESHOLD}']}/{row[f'Oui/Non <={AGE_THRESHOLD}']}), %yes >=48: {round(row[f'Oui >{AGE_THRESHOLD}'])}%. Objet: {row['Objet']} ({row['Date']})"
            ),
            axis=1,
        )
        party_delta[party_name] = dfv
    else:
        print("No councillor < 48 => no comparison")


UDC:
 54 >= 48
 26 < 48

Top 2 objects with largest gap:
Delta: 32.8 percent points, %yes <48: 50% (10/20), %yes >=48: 5%. Objet: Stop à l’îlot de cherté – pour des prix équitables. Initiative populaire et contre-projet indirect (2021-03-19 09:20:13)
Delta: 27.1 percent points, %yes <48: 47% (8/17), %yes >=48: 6%. Objet: Mariage civil pour tous (2020-12-18 08:45:07)
Delta: 25.3 percent points, %yes <48: 29% (4/14), %yes >=48: 21%. Objet: Double imposition. Convention avec le Liechtenstein (2016-06-17 09:44:40)
Delta: 20.1 percent points, %yes <48: 6% (1/16), %yes >=48: 5%. Objet: Doubles impositions. Convention avec la Nouvelle-Zélande (2020-06-19 10:21:28)
Delta: 19.6 percent points, %yes <48: 8% (1/13), %yes >=48: 9%. Objet: Double imposition. Convention avec la Norvège (2016-06-17 09:45:26)

PSS:
 40 >= 48
 24 < 48

Top 2 objects with largest gap:
Delta: 33.3 percent points, %yes <48: 71% (5/7), %yes >=48: 8%. Objet: Pour un revenu de base inconditionnel. Initiative populaire (2015

In [9]:
# Results for 6 political parties with enough <40 y.o. councillors
party_delta.keys()

dict_keys(['UDC', 'PSS', 'PLR', 'VERT-E-S', 'M-E', 'pvl', 'PDC'])

In [10]:
# Example usage
party_delta['PSS'].sort_values('Différence (%)', ascending=False).head(20).round(1)

,Oui >48,Oui <=48,Oui/Non >48,Oui/Non <=48,Oui >48 (%),Oui <=48 (%),Différence (%),Date,Objet
4,8,5,21,7,38.1,71.4,33.3,2015-12-18 08:18:29,Pour un revenu de base inconditionnel. Initiat...
87,28,7,28,10,100.0,70.0,30.0,2017-06-16 08:46:16,Ancrer durablement le taux spécial de TVA appl...
13,24,5,27,8,88.9,62.5,26.4,2016-03-18 09:12:25,Loi sur la surveillance de la correspondance p...
144,27,7,27,9,100.0,77.8,22.2,2018-09-28 10:47:10,Projet fiscal 17
155,8,3,21,17,38.1,17.6,20.4,2020-05-06 13:07:51,Révision partielle urgente de la loi fédérale ...
116,4,0,20,8,20.0,0.0,20.0,2018-03-16 08:20:49,Pour des denrées alimentaires saines et produi...
120,9,3,16,7,56.2,42.9,13.4,2018-03-16 08:26:45,Pour la souveraineté alimentaire. L’agricultur...
262,19,15,19,17,100.0,88.2,11.8,2021-10-01 08:49:49,Adaptation de l'âge limite en vigueur au sein ...
124,3,0,27,10,11.1,0.0,11.1,2018-03-16 08:38:01,Pour la promotion des voies cyclables et des c...
114,3,0,29,10,10.3,0.0,10.3,2017-12-15 09:35:42,Décision de l’OMC en matière de concurrence à ...


In [11]:
pd.options.display.max_colwidth = 200

In [12]:
# UDC
df_party = party_delta['UDC'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Stop à l’îlot de cherté – pour des prix équitables. Initiative populaire et contre-projet indirect,32.8,50.0,17.2,10,20,5,29,2021-03-19 09:20:13
,Mariage civil pour tous,27.1,47.1,20.0,8,17,6,30,2020-12-18 08:45:07
,Double imposition. Convention avec le Liechtenstein,25.3,28.6,53.8,4,14,21,39,2016-06-17 09:44:40
,Doubles impositions. Convention avec la Nouvelle-Zélande,20.1,6.2,26.3,1,16,5,19,2020-06-19 10:21:28
,Double imposition. Convention avec la Norvège,19.6,7.7,27.3,1,13,9,33,2016-06-17 09:45:26
,Loi sur les amendes d‘ordre,18.4,76.9,95.3,10,13,41,43,2016-03-18 09:20:11
,Loi sur les jeux d’argent,18.3,14.3,32.6,2,14,15,46,2017-09-29 08:56:28
,Exonérer les enfants du paiement des primes d'assurance-maladie,17.8,80.0,97.8,12,15,44,45,2017-03-17 08:41:30
,Développement de l'acquis de Schengen. Approbation et mise en oeuvre des échanges de notes entre la Suisse et l’UE concernant la reprise des règlements (UE) 2019/817 et 2019/818 relatifs à l’établ...,17.1,70.0,87.1,14,20,27,31,2021-03-19 09:30:18
,Améliorer la capacité d'action du Parlement en situation de crise,16.7,50.0,33.3,9,18,9,27,2022-03-18 09:12:56


In [13]:
# PSS
df_party = party_delta['PSS'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Pour un revenu de base inconditionnel. Initiative populaire,33.3,71.4,38.1,5,7,8,21,2015-12-18 08:18:29
,Ancrer durablement le taux spécial de TVA applicable à l'hébergement,30.0,70.0,100.0,7,10,28,28,2017-06-16 08:46:16
,Loi sur la surveillance de la correspondance par poste et télécommunication. Modification,26.4,62.5,88.9,5,8,24,27,2016-03-18 09:12:25
,Projet fiscal 17,22.2,77.8,100.0,7,9,27,27,2018-09-28 10:47:10
,Révision partielle urgente de la loi fédérale sur l'aviation face à la crise Covid-19,20.4,17.6,38.1,3,17,8,21,2020-05-06 13:07:51
,Pour des denrées alimentaires saines et produites dans des conditions équitables et écologiques (initiative pour des aliments équitables). Initiative populaire,20.0,0.0,20.0,0,8,4,20,2018-03-16 08:20:49
,Pour la souveraineté alimentaire. L’agriculture nous concerne toutes et tous. Initiative populaire,13.4,42.9,56.2,3,7,9,16,2018-03-16 08:26:45
,Adaptation de l'âge limite en vigueur au sein du Ministère public de la Confédération,11.8,88.2,100.0,15,17,19,19,2021-10-01 08:49:49
,Pour la promotion des voies cyclables et des chemins et sentiers pédestres (initiative vélo). Initiative populaire,11.1,0.0,11.1,0,10,3,27,2018-03-16 08:38:01
,Décision de l’OMC en matière de concurrence à l’exportation. Suppression des contributions à l’exportation pour les produits agricoles transformés,10.3,0.0,10.3,0,10,3,29,2017-12-15 09:35:42


In [14]:
# PLR
df_party = party_delta['PLR'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Charte européenne de l'autonomie locale. Protocole additionnel,42.3,28.6,70.8,2,7,17,24,2017-03-17 08:54:54
,Transformation et extension des réseaux électriques. Loi,39.1,0.0,39.1,0,5,9,23,2017-12-15 09:31:44
,Loi sur les jeux d’argent,33.3,57.1,90.5,4,7,19,21,2017-09-29 08:56:28
,Encouragement de la culture pour la période de 2021 à 2024,31.2,50.0,81.2,5,10,13,16,2021-10-01 08:41:48
,Aides financières à l’accueil extra-familial pour enfants. Modification,29.8,42.9,13.0,3,7,3,23,2017-06-16 08:47:48
,Projet fiscal 17,28.8,66.7,95.5,4,6,21,22,2018-09-28 10:47:10
,Organisation internationale du Travail. Protocole à la convention no 29 concernant le travail forcé,27.5,60.0,87.5,3,5,21,24,2017-03-17 08:53:49
,Echange international automatique de renseignements en matière fiscale. Loi,25.0,75.0,100.0,3,4,22,22,2015-12-18 08:24:33
,Assistance administrative fiscale. Convention du Conseil de l’Europe et de l’OCDE. Approbation,25.0,75.0,100.0,3,4,22,22,2015-12-18 08:25:34
,Blocage et restitution des avoirs illicites de personnes politiquement exposées à l’étranger. Loi,20.0,80.0,100.0,4,5,23,23,2015-12-18 08:17:31


In [15]:
# VERT-E-S
df_party = party_delta['VERT-E-S'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Pour un revenu de base inconditionnel. Initiative populaire,66.7,33.3,100.0,1,3,4,4,2015-12-18 08:18:29
,Projet fiscal 17,40.0,0.0,40.0,0,4,2,5,2018-09-28 10:47:10
,Pour une monnaie à l’abri des crises: émission monétaire uniquement par la Banque nationale! (Initiative Monnaie pleine). Initiative populaire,33.3,66.7,100.0,2,3,7,7,2017-12-15 09:32:36
,Loi sur l'imposition des huiles minérales. Révision partielle,16.7,0.0,16.7,0,3,1,6,2016-03-18 09:20:55
,Oui à l’interdiction de se dissimuler le visage. Initiative populaire et contre-projet indirect,15.9,9.1,25.0,1,11,2,8,2020-06-19 10:04:48
,Pour une eau potable propre et une alimentation saine - Pas de subventions pour l'utilisation de pesticides et l'utilisation d'antibiotiques à titre prophylactiques. Initiative populaire,6.7,6.7,0.0,1,15,0,10,2020-09-25 09:45:17
,LAVS. Modification (Utilisation systématique du numéro AVS par les autorités),3.3,6.7,10.0,1,15,1,10,2020-12-18 08:49:07
,Loi COVID-19. Modification,0.0,100.0,100.0,16,16,10,10,2020-12-18 08:57:41
,Indemnité forfaitaire octroyée pour la taxe sur la valeur ajoutée perçue sur la redevance de réception de radio et de télévision. Loi,0.0,100.0,100.0,15,15,10,10,2020-09-25 09:55:38
,LACI. Financement additionnel de l'assurance-chômage,0.0,100.0,100.0,15,15,9,9,2020-09-25 10:03:18


In [16]:
# M-E
df_party = party_delta['M-E'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Loi sur le service civil. Modification,43.4,75.0,31.6,6,8,6,19,2020-06-19 10:03:06
,CC. Droit de l’adoption. Modification,35.9,33.3,69.2,1,3,9,13,2016-06-17 09:17:29
,Révision de l’imposition à la source du revenu de l’activité lucrative. Loi,33.3,66.7,100.0,2,3,15,15,2016-12-16 08:17:15
,Développement de la production d'électricité d'origine hydraulique. Revoir la situation de référence des études d'impact,28.9,50.0,78.9,3,6,15,19,2019-12-20 08:58:20
,Charte européenne de l'autonomie locale. Protocole additionnel,19.0,66.7,85.7,2,3,12,14,2017-03-17 08:54:54
,Oui à la protection des enfants et des jeunes contre la publicité pour le tabac (enfants et jeunes sans publicité pour le tabac). Initiative populaire,15.0,80.0,95.0,4,5,19,20,2021-10-01 08:45:45
,CO. Droit de la société anonyme,14.3,85.7,100.0,6,7,20,20,2020-06-19 09:56:05
,Loi sur les stupéfiants. Modification (Médicaments à base de cannabis),14.3,85.7,100.0,6,7,18,18,2021-03-19 09:27:15
,Encouragement de la culture pour la période de 2021 à 2024,12.8,71.4,84.2,5,7,16,19,2021-10-01 08:41:48
,Pour plus de transparence dans le financement de la vie politique (initiative sur la transparence). Initiative populaire,12.5,87.5,100.0,7,8,19,19,2021-06-18 08:27:38


In [17]:
# PVL
df_party = party_delta['pvl'].sort_values('Différence (%)', ascending=False).head(10).round(1)
df_party = df_party[['Objet', 'Différence (%)', 'Oui <=48 (%)', 'Oui >48 (%)', 'Oui <=48', 'Oui/Non <=48', 'Oui >48', 'Oui/Non >48', 
        'Date']]
blankIndex=[''] * len(df_party)
df_party.index=blankIndex
df_party

,Objet,Différence (%),Oui <=48 (%),Oui >48 (%),Oui <=48,Oui/Non <=48,Oui >48,Oui/Non >48,Date
,Loi sur les travailleurs détachés. Modification,66.7,100.0,33.3,2,2,1,3,2016-09-30 09:09:48
,Relever l'examen de contrôle périodique effectué par un médecin-conseil de 70 à 75 ans pour les conducteurs âgés,50.0,50.0,100.0,1,2,5,5,2017-09-29 08:57:36
,Pour une Suisse libre de pesticides de synthèse. Initiative populaire,42.9,0.0,42.9,0,5,3,7,2020-09-25 09:46:41
,CO. Droit de la société anonyme,32.7,40.0,72.7,2,5,8,11,2020-06-19 09:56:05
,Entreprises responsables – pour protéger l’être humain et l’environnement. Initiative populaire,30.0,0.0,30.0,0,5,3,10,2020-06-19 09:58:36
,Décision de l’OMC en matière de concurrence à l’exportation. Suppression des contributions à l’exportation pour les produits agricoles transformés,20.0,0.0,20.0,0,1,1,5,2017-12-15 09:35:42
,Loi sur la surveillance de la correspondance par poste et télécommunication. Modification,20.0,100.0,80.0,2,2,4,5,2016-03-18 09:12:25
,Oui à la protection des enfants et des jeunes contre la publicité pour le tabac (enfants et jeunes sans publicité pour le tabac). Initiative populaire,20.0,0.0,20.0,0,5,2,10,2021-10-01 08:45:45
,Prestation transitoire pour les chômeurs âgés. Loi,10.0,100.0,90.0,5,5,9,10,2020-06-19 10:13:50
,Pour des soins infirmiers forts (initiative sur les soins infirmiers). Initiative populaire,10.0,100.0,90.0,4,4,9,10,2021-06-18 08:28:32
